In [100]:
import sys
import os
module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path.append('/Users/lex/miniconda2/lib/python2.7/site-packages')
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,TimeDistributed,Flatten,Masking
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import util.util as util

In [101]:
df_rf = pd.read_csv('rf_prediction.csv')
df_market = pd.read_csv('sp_prediction.csv')
df_fb = pd.read_csv('stock_predictions.csv')

In [102]:
df_fb['avg_price'] = (df_fb['open'] + df_fb['high'] + df_fb['low'])/3

In [103]:
df_fb = df_fb[~df_fb.ticker.isin(['APRN', 'SNAP'])]

In [104]:
test = df_fb.pivot_table('avg_price', 'date', 'ticker')

In [105]:
test.reset_index( drop=False, inplace=True )
test = test.fillna(0)

In [106]:
test['date'] = pd.to_datetime(test['date'])
test = test.sort_values('date')

In [107]:
corr_df = test.corr()
corr_matrix = test.corr().as_matrix()

In [108]:
percent_90d = test[test.columns[1:]].pct_change(90).dropna()

In [109]:
len(percent_90d.columns)

120

In [110]:
df_market['avg'] = (df_market['open'] + df_market['low'] + df_rf['close'])/3

In [111]:
df_market.tail()

,Unnamed: 0,open,high,low,close,avg
1342,2018-07-08,2693.851175,2694.378409,2596.571358,-1.805734e+09,1764.219844
1343,2018-07-09,2698.666616,2698.529588,2600.249846,-2.035934e+09,1767.049487
1344,2018-07-10,2698.162502,2698.325676,2599.866418,-1.709027e+09,1766.761973
1345,2018-07-11,2694.839657,2698.426951,2595.080368,-2.342949e+09,1764.084008
1346,2018-07-12,2694.087886,2697.560323,2596.243950,-2.200974e+09,1764.207945


In [112]:
def find_std_dict(df):
    std_dict = {}
    for x in df.columns:
        std = np.std(df[x])
        std_dict[x] = std
    return std_dict

def find_returns_list(df):
    returns_list = []
    for x in df.columns:
        avg = np.average(df[x])
        returns_list.append(avg)
    return returns_list

In [113]:
dict_90d = find_std_dict(percent_90d)

In [114]:
avg_return = find_returns_list(percent_90d)

In [115]:
len(dict_90d)

120

In [116]:
percent_180d = test[test.columns[1:]].pct_change(180).dropna(0)

In [117]:
dict_180d = find_std_dict(percent_180d.dropna())

In [118]:
stocks = list(percent_90d.columns.values)

In [119]:
import math
def varCovarMatrix(stocksInPortfolio, c, stdv):
    cm = np.array(c)
    vcv = []
    for eachStock in stocksInPortfolio:
        row = []
        for ticker in stocksInPortfolio:
            if eachStock == ticker:
                variance = math.pow(stdv[ticker],2)
                row.append(variance)
            else:
                cov = stdv[ticker]*stdv[eachStock]* cm[stocksInPortfolio.index(ticker)][stocksInPortfolio.index(eachStock)]
                row.append(cov)
        vcv.append(row)

    vcvmat = np.mat(vcv)

    return vcvmat

In [120]:
covariance_matrix_stocks = varCovarMatrix(stocks, corr_matrix, dict_90d)

In [121]:
def create_starting_weights(return_list):
    w0 = []
    for x in return_list:
        w0.append(1.0/len(return_list))
    return w0

def create_binaries(return_list):
    b0 = []
    for x in return_list:
        b0.append(0.0)
    return b0

In [122]:
covariance_matrix_stocks = np.nan_to_num(covariance_matrix_stocks)

In [123]:
avg_return[28] = 0

In [124]:
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt
from numpy.linalg import inv,pinv
from scipy.optimize import minimize

# USER INPUT
V = covariance_matrix_stocks  # covariance
R = np.reshape(np.matrix(avg_return), (120,1))
rf = 3/100

w0= create_starting_weights(avg_return)
y0 = [0] * len(w0)
# min var optimization
def calculate_portfolio_var(w,V):
    w0_matrix = np.array(w0)
    w = np.matrix(w) 
    returns = w * R
    return ((w*V*w.T)[0,0]) + 1/returns

# unconstrained portfolio (only sum(w) = 1 )
cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x)-1.0},
       {'type': 'ineq', 'fun': lambda x: x - 0.10})
bnds = []
#w0.append(y_matrix)
for x in w0:
    bnds.append((0,None))
bnds = tuple(bnds)
res= minimize(calculate_portfolio_var, w0, args=V, method='SLSQP',constraints=cons, bounds=bnds)
w_g = res.x
mu_g = w_g*R
var_g = np.dot(w_g*V,w_g)

In [126]:
res.x.shape

(120,)

In [127]:
V.shape

(120, 120)

In [79]:
mu_g

matrix([[ 0.06419776]])

In [945]:
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt
from numpy.linalg import inv,pinv
from scipy.optimize import minimize

# USER INPUT

w = np.array([1, 1, 1, 1])
y = np.array([1, 1, 1, 1])
v = [w,y]

def calculate_portfolio_var(v):
    w = v[0]
    y = v[1]
    return w*y*5

# unconstrained portfolio (only sum(w) = 1 )
cons = ({'type': 'eq', 'fun': lambda xy:  np.sum(xy[0])-1})
bnds = ((0, 1), (0,1), (0,1),(0,1),(0, None), (0, None), (0, None), (0, None))
res= minimize(calculate_portfolio_var, v, method='SLSQP',constraints=cons, bounds=bnds)
w_g = res.x
#mu_g = w_g*R
#var_g = np.dot(w_g*V,w_g)

In [946]:
w_g

array([ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.])

In [1]:
import itertools